# Data Acquisition - Lyrics

Selecting various artists that I like to listen to, along with some that I know like to say their own name in songs, I will be using the lyricsgenius library to call the Genius API to pull songs for that artist. From this I will gather name, title, year released and lyrics of the songs from each artist.

### Imports

In [1]:
import pandas as pd
import numpy as np

import time   
import lyricsgenius # Lyricsgenious Library to use with Genius API
from requests.exceptions import Timeout # To avoid timeout errors during pull

In [3]:
df = pd.read_csv('./data/gathering_lyrics.csv', index_col = 0)

In [4]:
df.drop('Unnamed: 0.1', axis = 1, inplace = True) # I did the data gathering with two notebooks to make the process faster.

In [5]:
df

,Artist,Featured,Title,release_year,lyrics
0,Jason Derulo,Jason Derulo,Swalla,2017-02-24,[Intro: Nicki Minaj]\nDrank\nYoung Money\n\n[V...
1,Jason Derulo,Jason Derulo,Talk Dirty,2013-08-02,"[Intro: Jason Derulo & Rie Abe]\n(Jason, haha\..."
2,Jason Derulo,Jason Derulo,Wiggle,2014-06-06,"[Intro: Jason Derulo & Snoop Dogg]\nAyo, Jason..."
3,Jason Derulo,Jason Derulo,Trumpets,2013-11-07,[Chorus]\nEvery time that you get undressed\nI...
4,Jason Derulo,Jason Derulo,Tip Toe,2017-11-10,[Intro: Jason Derulo & Soaky Siren]\nDerulo\nW...
...,...,...,...,...,...
11696,21 Savage,21 Savage,Hold Up,2018-04-20,"[Intro]\n21, 21\n21, 21\nGood head make a nigg..."
11697,21 Savage,21 Savage,Hollow Tips (Freestyle),2016-09-17,[Hook]\nI got a lot of extended clips\nLot of ...
11698,21 Savage,21 Savage,Act A Fool,NaN,"[Intro]\nWheezy Beats\nInternet, internet\nFuc..."
11699,21 Savage,21 Savage,Pass Her,NaN,"[Intro]\nGood job, First\n\n[Chorus]\nAll the..."


## TO DO:
- Check Dups
- clean strings
- Alias Column?
- Target Column
- Balance
- Popularity? Explicity?


In [8]:
df[df.duplicated('lyrics')].sort_values('lyrics')

,Artist,Featured,Title,release_year,lyrics
10398,ZAYN,ZAYN,Windowsill,NaN,\n Lyrics for this song h...
2911,Spencer Crandall,Spencer Crandall,Anything Outside,2020-08-14,\n Lyrics for this song h...
2897,Spencer Crandall,Spencer Crandall,Things I Can’t Say,2020-09-11,\n Lyrics for this song h...
6734,Dirty South,Dirty South,The End (Tocadisco Loves Techno Vocal Mix),NaN,\n Lyrics for this song h...
10591,Sam Smith,Sam Smith,Forgive Myself,2020-10-30,\n Lyrics for this song h...
...,...,...,...,...,...
11098,Clean Bandit,Clean Bandit,Rockabye (No Rap),NaN,"[Verse 1]\nShe works the nights, by the water\..."
8632,Armin van Buuren,Armin van Buuren,Freefall (Heatbeat Remix),NaN,[Verse 1]\nWe're dodging daggers that almost r...
2378,Lorde,Lorde,“Yellow Flicker Beat” Single Art,2014-09-29,NaN
4866,The Proclaimers,The Proclaimers,After You’re Gone,NaN,NaN


In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 100

In [ ]:
df.sort_values('lyrics')

In [ ]:
len(artist_name)

In [ ]:
len(lyrics)

In [ ]:
len()